# AI-Powered HR Assistant: A Use Case for Nestle’s HR Policy Documents

## Nestlé HR Policy Chatbot Overview

This Jupyter Notebook implements a conversational chatbot designed to answer user queries based on information contained within Nestlé's HR policy documents. It leverages several powerful technologies from the field of Natural Language Processing (NLP) and Large Language Models (LLMs) to achieve this:

1.  **Document Loading and Chunking:** The notebook begins by loading the HR policy document (in PDF format) and splitting it into smaller, manageable text chunks. This is crucial because LLMs have limitations on the amount of text they can process at once.

2.  **Text Embeddings:** Each text chunk is then converted into a numerical vector representation called an "embedding." These embeddings capture the semantic meaning of the text, allowing the system to understand the relationships between different pieces of information. OpenAI's embedding models are used for this purpose.

3.  **Vector Database:** The embeddings are stored in a vector database (Chroma), which allows for efficient similarity search. This means that when a user asks a question, the system can quickly find the most relevant text chunks from the HR policy.

4.  **Question Answering with LLM:** The most relevant text chunks are then passed to a large language model (OpenAI's GPT model) along with the user's question. The LLM uses this context to generate a coherent and informative answer.

5.  **User Interface:** Finally, a user-friendly interface is created using Gradio, allowing users to easily interact with the chatbot.

In summary, this notebook demonstrates a complete workflow for building a question-answering system over a PDF document using state-of-the-art NLP and LLM techniques. This approach can be generalized to other document types and domains, making it a valuable tool for information retrieval and knowledge management. The notebook is structured with Markdown explanations and code blocks separated by functionality, making it easy to follow and understand the implementation details.


---


### Installation of Libraries

This code block uses `pip` to install the necessary Python libraries. These libraries are essential for the chatbot's functionality:

- `openai`: For interacting with OpenAI's models (GPT).
- `langchain`: A framework for developing applications powered by language models.
- `chromadb`: A vector database for storing and retrieving text embeddings.
- `pypdf`: For loading and processing PDF documents.
- `gradio`: For creating the user interface.
- `tiktoken`: For tokenizing text, especially important for managing context windows with large language models.

**Note:** You only need to run this cell once. If you've already installed these libraries, you can skip this step. If you are running this in a Google Colab notebook, you will need to run this section every time you connect to a new runtime.


In [1]:
# Install necessary libraries (run this in your notebook environment if needed)
# !pip install openai langchain chromadb pypdf gradio tiktoken

### Import Libraries and Set API Key

This block imports the required libraries and sets up the OpenAI API key.

- **Imports:** Imports the necessary classes and functions from the installed libraries.
- **API Key:** Sets the OpenAI API key using environment variables. This is the **recommended** way to manage API keys for security reasons.


In [2]:
import os
import logging

from dotenv import load_dotenv

import openai

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

import gradio as gr

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Set OpenAI API key (replace with your actual key - use environment variables)
def load_env():
    """Load and validate environment variables"""
    load_dotenv(verbose=True)
    api_key = os.getenv('OPENAI_API_KEY')
    pdf_doc = os.getenv('PDF_DOC_PATH')
    if not api_key:
        raise ValueError("OPENAI_API_KEY not found in environment variables")
    if not pdf_doc:
        raise ValueError("PDF_DOC_PATH note found in environment variables")
    return api_key, pdf_doc

api_key, pdf_doc = load_env()

/opt/homebrew/anaconda3/envs/1720690009_20241228_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the PDF Document

This block loads the PDF document using `PyPDFLoader`.

- **`PyPDFLoader`:** This class from `langchain` loads the PDF file.
- **`loader.load()`:** This method reads the PDF and extracts the text content.
- **Error Handling:** The `try-except` block handles the case where the specified PDF file is not found, preventing the code from crashing.


In [3]:
# Load the PDF document
try:
    loader = PyPDFLoader(pdf_doc) # Replace with your PDF file name
    documents = loader.load()
except FileNotFoundError:
    print("Error: {pdf_doc} not found. Please ensure the file is in the correct directory or provide the correct path.")
    exit()

### Split the Document into Chunks

Large documents are often split into smaller chunks for processing by language models. This block uses `CharacterTextSplitter` to do this.

- **`CharacterTextSplitter`:** This class splits the text into chunks of a specified size.
- **`chunk_size`:** The maximum number of characters in each chunk (1000 in this case).
- **`chunk_overlap`:** The number of overlapping characters between adjacent chunks (0 in this case). Overlapping chunks can help maintain context.
- **`texts = text_splitter.split_documents(documents)`:** This line performs the actual splitting.


In [4]:
# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

### Create Embeddings and Store in Chroma

This block creates vector embeddings of the text chunks and stores them in a Chroma vector database.

*   **`OpenAIEmbeddings()`:** This creates an embedding model from OpenAI, which converts text into numerical vectors.
*   **`Chroma.from_documents(texts, embeddings)`:** This creates a Chroma database and adds the text embeddings to it. The embeddings allow for efficient semantic search.

In [5]:
# Create embeddings and store them in Chroma
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)

2024-12-29 14:45:18,275 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-12-29 14:45:19,523 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### Create a RetrievalQA Chain

This block creates the question-answering chain using `RetrievalQA`.

*   **`RetrievalQA`:** This chain combines a language model with a retriever (in this case, our Chroma database) to answer questions based on the retrieved information.
*   **`llm=OpenAI(temperature=0)`:** Specifies the OpenAI language model to use. `temperature=0` makes the responses more deterministic.
*   **`chain_type="stuff"`:** This chain type "stuffs" all the retrieved context into the prompt to the language model.
*   **`retriever=db.as_retriever()`:** Sets the retriever to our Chroma database.

In [6]:
# Create a RetrievalQA chain
qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=db.as_retriever())

/var/folders/1c/h_10706j24qc8qjh7x3yy0g40000gn/T/ipykernel_81788/2092767558.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=db.as_retriever())


### Define the Chatbot Function

This block defines the `chatbot` function, which takes a user query as input and returns the chatbot's response.

*   **`qa.run(query)`:** This runs the question-answering chain with the user's query.
*   **Error Handling:** The `try-except` block handles potential errors during query processing.

In [7]:
# Define the chatbot function
def chatbot(query):
    try:
        result = qa.run(query)
        return result
    except Exception as e:
        return f"An error occurred: {e}"

### Create and Launch the Gradio Interface

This block creates and launches the Gradio user interface.

*   **`gr.Interface(...)`:** Creates the Gradio interface.
*   **`fn=chatbot`:** Specifies the function to call when the user submits a query.
*   **`inputs`:** Defines the input component (a textbox).
*   **`outputs`:** Defines the output component (another textbox).
*   **`title` and `description`:** Set the title and description of the interface.
*   **`iface.launch()`:** Launches the Gradio interface, making it accessible in your browser.

In [8]:
# Create the Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query about Nestlé's HR policy here..."),
    outputs=gr.Textbox(lines=5),
    title="Nestlé HR Chatbot",
    description="Ask me anything about Nestlé's HR policies.",
)

# Launch the Gradio interface
iface.launch()


2024-12-29 14:45:19,844 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-29 14:45:19,853 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2024-12-29 14:45:20,061 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
/var/folders/1c/h_10706j24qc8qjh7x3yy0g40000gn/T/ipykernel_81788/2060248823.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)
2024-12-29 14:45:54,382 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-29 14:45:55,035 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-12-29 14:47:11,305 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-29 14:47:12,837 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-12-29 14:47:49,624 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-29 14:47:51,001 - httpx - INFO - HTTP Request: POST h